# Data Cleaning

In [27]:
# Import dependencies
import pandas as pd

In [28]:
# Store file names, desired areas, and desired states in lists
file_names = ["lat_lon", "population", "commute",\
             "atlanta_housing", "chicago_housing", "dallas_housing", "denver_housing"]
areas = ["Atlanta", "Chicago", "Dallas", "Denver", "Detroit", "Los Angeles", "Miami", "New York", "Philadelphia"]
states = ["GA", "IL", "TX", "CO", "MI", "CA", "FL", "NY", "PA"]

#areas = ["Atlanta", "Boston", "Chicago", "Dallas", "Denver", "Los Angeles", "New York", "St Louis"]
#states = ["GA", "MA", "IL", "TX", "CO", "CA", "NY", "MO"]

areas = sorted(areas)

In [29]:
# define function to read csv files:
def get_file():
    global df
    data = pd.read_csv(f"resources/{file_name}.csv", encoding="ISO-8859-1", header=None, error_bad_lines=False)  
    df = pd.DataFrame(data)
    print(f"\nraw: {file_name}")
    return df

# Latitudes and Longitudes

In [30]:
# extract latitudes and longitudes
file_name = "lat_lon"

# Get raw data from all files and print out
get_file()
df = df.reset_index()
df = df.loc[2:, [2,3,5,6]]
columns = ["state", "area", "latitude", "longitude"]
df.columns = columns

# Get desired data from DataFrame:
df_list = pd.DataFrame(columns=columns, data=None)

# Loop through area list and state list to get data
for i in range(len(areas)):
    df_i = df.loc[(df["area"]==areas[i]) & (df["state"]==states[i])]
    df_i = df_i.groupby("area").first().reset_index()
    df_list = df_list.append(df_i)
df = pd.DataFrame(columns=columns, data=df_list)

# Adjust display of data
df = df.iloc[:(len(areas)+1),:]
df = df.reset_index(drop=True)
print(f"\n\nclean: {file_name}")
print("\n\nfinal:")
mega_df = df
print(mega_df)


raw: lat_lon


clean: lat_lon


final:
  state          area  latitude  longitude
0    GA       Atlanta   33.8477   -84.2814
1    IL       Chicago   41.9288   -87.6315
2    TX        Dallas   32.7825   -96.8207
3    CO        Denver   39.7525  -104.9995
4    MI       Detroit   42.3679   -83.1386
5    CA   Los Angeles   34.0396  -118.2661
6    FL         Miami   25.9388   -80.2144
7    NY      New York   40.7528   -73.9725
8    PA  Philadelphia   39.9524   -75.1653


# Population

In [31]:
# Variables
file_name = "population"

# Get raw data from all files and print out
get_file()
df = df.iloc[3:(3+len(areas)), [2,3]]
print(df)

# Get data for desired areas
columns = ["metro", file_name]
df.columns = columns
df["area"] = df["metro"].str.split('-', 1).str[0]
del df["metro"]
columns = ["area", file_name]
df = df[columns].sort_values("area")
df = df.reset_index(drop=True)
df = df.dropna()
print(f"\n\nclean: {file_name}")
print(df)

mega_df = pd.concat([mega_df, df], axis=1)
mega_df = mega_df.T.drop_duplicates().T
print("\n\nfinal:")
print(mega_df)


raw: population
                                                    2         3
3   Los Angeles-Long Beach-Anaheim, CA Metro Area;...  13189366
4     Denver-Aurora-Lakewood, CO Metro Area; Colorado   2752056
5   Miami-Fort Lauderdale-West Palm Beach, FL Metr...   5926955
6   Atlanta-Sandy Springs-Roswell, GA Metro Area; ...   5612777
7   Chicago-Naperville-Elgin, IL-IN-WI Metro Area ...   8656303
8    Detroit-Warren-Dearborn, MI Metro Area; Michigan   4296731
9   New York-Newark-Jersey City, NY-NJ-PA Metro Ar...  13380318
10  Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Me...   4076378
11  Dallas-Fort Worth-Arlington, TX Metro Area; Texas   6957123


clean: population
           area population
0       Atlanta    5612777
1       Chicago    8656303
2        Dallas    6957123
3        Denver    2752056
4       Detroit    4296731
5   Los Angeles   13189366
6         Miami    5926955
7      New York   13380318
8  Philadelphia    4076378


final:
  state          area latitude longitude pop

# Commute

In [32]:
# Define function to extract latitudes and longitudes
def commute():
    file_name = "commute"
    global df
    global mega_df
    # Get raw data from all files and print out
    get_file()
    df = df.loc[2:, [2,3]]
    print(df)
    # Get data for desired areas
    columns = ["metro", file_name]
    df.columns = columns
    df.loc[range(len(areas)), columns]
    df["area"] = df["metro"].str.split('-', 1).str[0]
    del df["metro"]
    columns = ["area", file_name]
    df = df[columns].sort_values("area")
    df = df.reset_index(drop=True)
    df = df.loc[range(len(areas)), :]
    print(f"\n\nclean: {file_name}")
    print(df)
    mega_df = pd.concat([mega_df, df], axis=1)
    mega_df = mega_df.T.drop_duplicates().T
    
    # Add calucation column to mega_df
    values = mega_df["commute"].astype(float)/mega_df["population"].astype(float)
    mega_df["commute/person"] = values
    del(mega_df["commute"])
    print("\n\nfinal:")
    print(mega_df)

In [33]:
# Global variables
file_name = "commute"

# Call functions and get latitudes and longitudes
commute()


raw: commute
                                                    2          3
2   Los Angeles-Long Beach-Anaheim, CA Metro Area;...  171087545
3     Denver-Aurora-Lakewood, CO Metro Area; Colorado   36188125
4   Miami-Fort Lauderdale-West Palm Beach, FL Metr...   74035650
5   Atlanta-Sandy Springs-Roswell, GA Metro Area; ...   75949245
6   Chicago-Naperville-Elgin, IL-IN-WI Metro Area ...  125086155
7    Detroit-Warren-Dearborn, MI Metro Area; Michigan   49061055
8   New York-Newark-Jersey City, NY-NJ-PA Metro Ar...  225622415
9   Philadelphia-Camden-Wilmington, PA-NJ-DE-MD Me...   54031170
10  Dallas-Fort Worth-Arlington, TX Metro Area; Texas   88686850


clean: commute
           area    commute
0       Atlanta   75949245
1       Chicago  125086155
2        Dallas   88686850
3        Denver   36188125
4       Detroit   49061055
5   Los Angeles  171087545
6         Miami   74035650
7      New York  225622415
8  Philadelphia   54031170


final:
  state          area latitude longitude

/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel/__main__.py:13: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike


# Gym numbers

In [34]:
mega_df.columns

Index(['state', 'area', 'latitude', 'longitude', 'population',
       'commute/person'],
      dtype='object')

In [40]:
# Save dataframe to file
mega_df.columns = ["state", "Metropolitan Area", "Latitude", "Longitude", "Population",\
                   "Average travel time to work of workers(minutes)"]
print(mega_df)
mega_df.to_csv("cleaning_output.csv")

  state Metropolitan Area Latitude Longitude Population  \
0    GA           Atlanta  33.8477  -84.2814    5612777   
1    IL           Chicago  41.9288  -87.6315    8656303   
2    TX            Dallas  32.7825  -96.8207    6957123   
3    CO            Denver  39.7525  -104.999    2752056   
4    MI           Detroit  42.3679  -83.1386    4296731   
5    CA       Los Angeles  34.0396  -118.266   13189366   
6    FL             Miami  25.9388  -80.2144    5926955   
7    NY          New York  40.7528  -73.9725   13380318   
8    PA      Philadelphia  39.9524  -75.1653    4076378   

   Average travel time to work of workers(minutes)  
0                                        13.531492  
1                                        14.450298  
2                                        12.747633  
3                                        13.149487  
4                                        11.418228  
5                                        12.971628  
6                                     